In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2     # actually reload imports
#%config InlineBackend.figure_format = 'svg'    # use vectorial figures
from smartg_view import semi_polar, smartg_view
from smartg import Smartg, RoughSurface, LambSurface, FlatSurface, Environment
from profile.profil import *
from luts import MLUT, LUT, Idx, merge, read_lut_hdf, read_mlut_hdf
from water.iop_spm import IOP_SPM
from water.iop_mm import IOP_MM
import ephem
from PAR import Int, Int2, Irr, ReadREPTRAN_bands, dailyPAR, viewPAR, ObsPAR, simulatePAR
import pickle

In [ ]:
fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, dt_l_l, SZA_l, raa_l = simulatePAR(CALC=False)
# store file lists
#with open('simulations_list_continental.pickle3', 'w') as f:
  #pickle.dump([fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, SZA_l, raa_l], f)
# recompute date list
fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, dt_l_l, SZA_l, raa_l = simulatePAR(CALC=False,verbose=True)
# Getting back the file lists:
with open('simulations_list_continental.pickle3') as f:
    fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, SZA_l, raa_l = pickle.load(f)

In [ ]:
fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, dt_l_l, SZA_l, raa_l=simulatePAR(CALC=False)

In [ ]:
# Getting back the objects:
with open('simulations_list_continental.pickle3') as f:
    fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, SZA_l, raa_l = pickle.load(f)

In [ ]:
ObsPAR(fpp2_l_l, fsp2_l_l, SZA_l, raa_l,verbose=True)

In [ ]:
dailyPAR(fpp_l_l,fsp_l_l,dt_l_l,verbose=True)

In [ ]:
with open('simulations_list_stratocumulus5.pickle') as f:
    fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, SZA_l, raa_l = pickle.load(f)

In [ ]:
ObsPAR(fpp2_l_l, fsp2_l_l, SZA_l, raa_l, verbose=True)

In [ ]:
dailyPAR(fpp_l_l,fsp_l_l,dt_l_l,verbose=True)

In [ ]:
with open('simulations_list_cirrus.pickle') as f:
    fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, SZA_l, raa_l = pickle.load(f)

In [ ]:
ObsPAR(fpp2_l_l, fsp2_l_l, SZA_l, raa_l)

In [ ]:
dailyPAR(fpp_l_l,fsp_l_l,dt_l_l,verbose=False)

In [ ]:
with open('simulations_list_stratocumulus50.pickle') as f:
    fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, SZA_l, raa_l = pickle.load(f)

In [ ]:
ObsPAR(fpp2_l_l, fsp2_l_l, SZA_l, raa_l,verbose=False)

In [ ]:
dailyPAR(fpp_l_l,fsp_l_l,dt_l_l,verbose=False)

In [ ]:
LMIN=400. # nm
LMAX=700. # nm
SAMPLING=1
PFRES=50. # spectral resolution for phase function (nm)
grid='100[30]10[5]10[1]0' # altitude grid for scattering and absorption coefficients
o3=300. # DU
wref=862. # wavelength for AOT and COT reference
aot=0.1
aermodel_l=['maritime_clean','continental_average']
aermodel=aermodel_l[1]
chl=0.1 # mg/l
ws=5. # m/s
aer = AeroOPAC(aermodel, aot, wref)
repname='reptran_solar_coarse' # k distribution
surf=RoughSurface(SUR=3, WIND=ws, NH2O=1.34)
wi,wb,we,ex,dl = ReadREPTRAN_bands(repname,LMIN=LMIN,LMAX=LMAX,SAMPLING=SAMPLING)
pfwav=np.linspace(LMIN, LMAX, num=(LMAX-LMIN)/PFRES+1, endpoint=True)
water=IOP_MM(chl,NANG=7201,pfwav=pfwav)
atm=Profile('afglmw',O3=o3,pfwav=pfwav
             #,cloud=CloudOPAC('strato',[('wc.sol.mie', 1, 10. , 9., 10.)], 5, 862.)
             ,aer=aer 
           )
             


In [ ]:
job = Smartg('SMART-G-PP', THVDEG=30, wl=list(wi.data), NBPHOTONS=1e7,
                    atm=atm,NBLOOP=5000, OUTPUT_LAYERS=3,
                    surf=surf, water=water,NFAER=10000,NFOCE=10000,
                    overwrite=True)

In [ ]:
# multispectral
chl=1 # mg/l
ws=1. # m/s
o3=300. # DU
surf=RoughSurface(SUR=3, WIND=ws, NH2O=1.34)
water=IOP_MM(chl,NANG=7201,pfwav=[680,700])
pro = Profile('afglmw',
              grid='100[20]20[5]10[1]0', # altitude grid for scattering and absorption coefficients
              pfgrid=[100, 20, 10., 5., 0.],   # optional, otherwise use a single band 100-0
              pfwav=[680,700], # optional, otherwise phase functions are calculated at all bands
              aer=AeroOPAC('continental_average', 0.1, 862.),O3=o3,
              verbose=False)
LMIN=680. # nm
LMAX=700. # nm
SAMPLING=1
repname='reptran_solar_fine' # k distribution
wi,wb,we,ex,dl,bands = ReadREPTRAN_bands(repname,LMIN=LMIN,LMAX=LMAX,SAMPLING=SAMPLING,FULL=True)
job = Smartg('SMART-G-PP', THVDEG=30, wl=bands, NBPHOTONS=1e9,
                    atm=pro,NBLOOP=5000, OUTPUT_LAYERS=3,
                    surf=surf, NFAER=10000,NFOCE=10000,
                    water=water,
                    overwrite=True,cmdfile='/home/did/RTC/SMART-G/zozo.txt')

In [ ]:
from luts import MLUT
M=job.read()
MM_l=[]
for k in range(len(M.luts)):
    MM_tmp = M.luts[k]
    MM=LUT(MM_tmp.data,axes=[wi.data,MM_tmp.axes[1],MM_tmp.axes[2]],desc=MM_tmp.desc,
           names=['Wavelengths',MM_tmp.names[1],MM_tmp.names[2]],attrs=MM_tmp.attrs)
    MM_l.append(MM)
M2 = MLUT(MM_l)
M2.print_info()
Ip_up0= M2['Q_up (0+)']
Ip_up0.data=np.sqrt(Ip_up0.data**2 + M2['U_up (0+)'].data**2)
Ip_up0.desc='Ip_up (0+)'
semi_polar(Ip_up0.sub[:,-1,:], swap=True, index=20,vmax=0.5)

In [ ]:
jobp = Smartg('SMART-G-PP', wl=400.,# wl = np.linspace(400, 700, 2),
             THVDEG=29.9, NBPHOTONS=1e9,NBTHETA=180,
             atm=pro,OUTPUT_LAYERS=3,NFAER=10000,NFOCE=10000,
             surf=RoughSurface(SUR=3, WIND=ws, NH2O=1.34),
             water=IOP_MM(chl,NANG=7201),
             overwrite=True)

In [ ]:
jobe = Smartg('SMART-G-SP', wl=400.,# wl = np.linspace(400, 700, 2),
             THVDEG=29.9, NBPHOTONS=1e8,NBTHETA=180,
             atm=pro,OUTPUT_LAYERS=3,NFAER=10000,NFOCE=10000,
             surf=RoughSurface(SUR=3, WIND=ws, NH2O=1.34),
             water=IOP_MM(chl,NANG=7201),env=Environment(ENV=1,ENV_SIZE=1000,X0=990,ALB=0.5),
             overwrite=True)

In [ ]:
fig = figure(figsize=(8,8))
M=jobe.read()
I_up = M['I_up (TOA)']
I_up0= M['I_up (0+)']
Ip_up = M['Q_up (TOA)']
Ip_up.data=np.sqrt(Ip_up.data**2 + M['U_up (TOA)'].data**2)
Ip_up.desc='Ip_up (TOA)'
Ip_up0= M['Q_up (0+)']
Ip_up0.data=np.sqrt(Ip_up0.data**2 + M['U_up (0+)'].data**2)
Ip_up0.desc='Ip_up (0+)'
semi_polar(I_up, index=Idx(2.), rect=421, sub=423, swap=False, fig=fig,vmin=0,vmax=0.5)
semi_polar(Ip_up, index=Idx(2.), rect=422, sub=424, swap=False, fig=fig,vmin=0,vmax=0.5)
semi_polar(I_up0, index=Idx(2.), rect=425, sub=427, swap=False, fig=fig,vmin=0,vmax=0.5) 
semi_polar(Ip_up0, index=Idx(2.), rect=426, sub=428, swap=False, fig=fig,vmin=0,vmax=0.5) 

In [ ]:
fig = figure(figsize=(8,8))
M=jobs.read()
I_up = M['I_up (TOA)']
I_up0= M['I_up (0+)']
Ip_up = M['Q_up (TOA)']
Ip_up.data=np.sqrt(Ip_up.data**2 + M['U_up (TOA)'].data**2)
Ip_up.desc='Ip_up (TOA)'
Ip_up0= M['Q_up (0+)']
Ip_up0.data=np.sqrt(Ip_up0.data**2 + M['U_up (0+)'].data**2)
Ip_up0.desc='Ip_up (0+)'
semi_polar(I_up, index=Idx(2.), rect=421, sub=423, swap=False, fig=fig,vmin=0,vmax=0.5)
semi_polar(Ip_up, index=Idx(2.), rect=422, sub=424, swap=False, fig=fig,vmin=0,vmax=0.5)
semi_polar(I_up0, index=Idx(2.), rect=425, sub=427, swap=False, fig=fig,vmin=0,vmax=0.5) 
semi_polar(Ip_up0, index=Idx(2.), rect=426, sub=428, swap=False, fig=fig,vmin=0,vmax=0.5) 